# Pré-processamento
Este notebook é responsável por pré-processar os dados realizando as seguintes tarefas:
- Tratamento de dados ausentes
- Definição de tipos
- Normalização e discretização
- Limpeza de dados (univariado, bivariado e multivariado)

Este notebook usa o 'clean_vehicles.csv' como dataset, que já não tem mais as colunas 'county', 'region_url' e 'image_url'.

In [151]:
from Utils import *

DATASET = "../datasets/clean_vehicles.csv"
df = pd.read_csv(DATASET)

## Definição de tipos
Olhando para a tabela que descreve o dataset em '1-initial-analysis.ipynb' e pela análise feita no notebook, os dados foram definidos nos formatos abaixo.

### Dados categóricos
- type
- region
- transmission
- manufacturer
- model
- condition
- cylinders
- fuel
- title_status
- drive
- size
- paint_color
- state

### Dados do tipo inteiro
- id
- price
- year

### Dados do tipo float
- odometer
- lat
- long

### Dados do tipo object
- url
- description
- vin

## Tratamento de dados ausentes ou nulos
Antes de podermos definir os tipos dos dados vamos primeiro lidar com os dados nulos ou ausentes.

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435849 entries, 0 to 435848
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            435849 non-null  int64  
 1   url           435849 non-null  object 
 2   region        435849 non-null  object 
 3   price         435849 non-null  int64  
 4   year          434732 non-null  float64
 5   manufacturer  415102 non-null  object 
 6   model         429650 non-null  object 
 7   condition     249043 non-null  object 
 8   cylinders     269465 non-null  object 
 9   fuel          432858 non-null  object 
 10  odometer      360701 non-null  float64
 11  title_status  434043 non-null  object 
 12  transmission  433703 non-null  object 
 13  vin           239197 non-null  object 
 14  drive         313838 non-null  object 
 15  size          139888 non-null  object 
 16  type          318741 non-null  object 
 17  paint_color   300602 non-null  object 
 18  desc

In [36]:
# Verifica quais dados sãbo nulos.
df.isnull().sum()

id                   0
url                  0
region               0
price                0
year              1117
manufacturer     20747
model             6199
condition       186806
cylinders       166384
fuel              2991
odometer         75148
title_status      1806
transmission      2146
vin             196652
drive           122011
size            295961
type            117108
paint_color     135247
description         27
state                0
lat               8235
long              8235
dtype: int64

### Tratamento de dados ausentes em Dados categóricos
Para os dados categóricos iremos criar uma nova categoria 'undefined' para substituir os dados ausentes.

In [153]:
categoric_types = ['type', 'region', 'transmission', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'drive', 'size', 'paint_color', 'state']

for column in categoric_types:
    df[column] = df[column].fillna('undefined')

In [154]:
# confirmando que não há mais amostras com dados categoricos nulos no novo dataframe
df.isnull().sum()

id                   0
url                  0
region               0
price                0
year              1117
manufacturer         0
model                0
condition            0
cylinders            0
fuel                 0
odometer         75148
title_status         0
transmission         0
vin             196652
drive                0
size                 0
type                 0
paint_color          0
description         27
state                0
lat               8235
long              8235
dtype: int64

### Tratamento de dados ausentes em Dados do tipo inteiro
Para os dados do tipo inteiro foi verificado que existe dados nulos apenas na coluna 'year', poucas amostras possui valor nulo na coluna 'year', representando 0,2% do dataset. Como o ano é um fator importante para a compra e venda de carros, queremos manter essas amostras.

Foi feita uma análise e notou-se que nesses casos o ano é na maioria das vezes os 4 primeiros dígitos da descrição seguidos da marca e modelo do carro. Usamos a função 'get_year_from_description' para realizar esse processo, a definição da função está no Utils.py.

In [155]:
# mostra os 20 primeiros caracteres da descrição das linhas que possuem a coluna ano nula
for elem in df[df['year'].isnull().values]['description']:
    try:
        print(elem[0:40])
    except:
        print(elem)

2019 *Chrysler* *Pacifica* Touring Plus 
2019 *Jeep* *Grand Cherokee* Laredo SUV 
2019 *Chevrolet* *Tahoe* 2WD 4dr LT SUV 
2018 *Toyota* *Tacoma* TACOMA SR5 V6 DOU
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2018 *Subaru* *Forester* 2.5i CVT SUV - 
2018 *Volkswagen* *Jetta* 1.4T S Automat
2018 *Cadillac* *CTS-V Sedan* 4dr Sedan 
2019 *Ford* *F-150* Lariat 4x4 4dr Super
2018 *Honda* *Accord Sedan* Sport 2.0T 1
2019 *Kia* *Optima* LX Automatic Sedan -
2020 *Chevrolet* *5500XD LCF Diesel* 550
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2019 *Chevrolet* *Corvette* 2dr Grand Sp
2018 *Chevrolet* *Tahoe* LT-4X4-22S-NEW 
2018 *Honda* *Pilot* EX-L 2WD SUV - $28,
2018 *Acura* *TLX*  Sedan - $29,975Call 
2019 *Chevrolet* *Silverado 1500 LD* LT 
2019 *Mitsubishi* *Mirage* ES CVT ONLY 3
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2018 *Subaru* *Legacy* 2.5i Premium Seda
2018 *Hyundai* *Accent* SE Sedan Automat
2018 *Toyota* *Tacoma* TACOMA SR5 V6 DOU
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2019 *BMW* *X7* 

In [156]:
get_year_from_description(df)

Ainda existem dados de ano nulos (28 linhas), analisando podemos perceber que muitas das descrições são nulas ou não estão no formato esperado e nesses casos muitos outros atributos estão undefined. Iremos remover os que tiverem a coluna 'description' nula.

In [157]:
df[df['year'].isnull()]


,id,url,region,price,year,manufacturer,model,condition,cylinders,fuel,...,transmission,vin,drive,size,type,paint_color,description,state,lat,long
15321,7115299820,https://chattanooga.craigslist.org/cto/d/ten-m...,chattanooga,2700,NaN,undefined,ep liberty,excellent,6 cylinders,gas,...,automatic,NaN,rwd,mid-size,SUV,grey,03 jeep liberty 2 wheel drive v6 runs good col...,tn,35.6279,-84.7542
28526,7119521844,https://wausau.craigslist.org/cto/d/rosholt-20...,wausau,4300,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,wi,NaN,NaN
40969,7119451639,https://worcester.craigslist.org/cto/d/worcest...,worcester / central MA,4500,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,ma,NaN,NaN
74356,7121504251,https://losangeles.craigslist.org/lgb/cto/d/la...,los angeles,2600,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,ca,NaN,NaN
74400,7121502508,https://losangeles.craigslist.org/lac/cto/d/la...,los angeles,2600,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,ca,NaN,NaN
80723,7119965827,https://honolulu.craigslist.org/oah/cto/d/waip...,hawaii,5200,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,hi,NaN,NaN
87185,7121258338,https://losangeles.craigslist.org/lac/cto/d/lo...,los angeles,6000,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,ca,NaN,NaN
87528,7120849219,https://greenville.craigslist.org/cto/d/lauren...,greenville / upstate,2000,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,sc,NaN,NaN
87633,7121247705,https://boise.craigslist.org/cto/d/riggins-200...,boise,8200,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,id,NaN,NaN
89520,7118786517,https://greenville.craigslist.org/cto/d/greenv...,greenville / upstate,8000,NaN,undefined,undefined,undefined,undefined,undefined,...,undefined,NaN,undefined,undefined,undefined,undefined,NaN,sc,NaN,NaN


In [160]:
df.dropna(subset=['description'], inplace=True)
df.isnull().sum()

id                   0
url                  0
region               0
price                0
year                 4
manufacturer         0
model                0
condition            0
cylinders            0
fuel                 0
odometer         75124
title_status         0
transmission         0
vin             196625
drive                0
size                 0
type                 0
paint_color          0
description          0
state                0
lat               8211
long              8211
dtype: int64

#### Sobram quatro linhas em que o ano é nulo:

In [168]:
df[df['year'].isnull()]

,id,url,region,price,year,manufacturer,model,condition,cylinders,fuel,...,transmission,vin,drive,size,type,paint_color,description,state,lat,long
15321,7115299820,https://chattanooga.craigslist.org/cto/d/ten-m...,chattanooga,2700,NaN,undefined,ep liberty,excellent,6 cylinders,gas,...,automatic,NaN,rwd,mid-size,SUV,grey,03 jeep liberty 2 wheel drive v6 runs good col...,tn,35.6279,-84.7542
170030,7121276517,https://sandiego.craigslist.org/nsd/cto/d/esco...,san diego,5000,NaN,undefined,yota 4x4,excellent,6 cylinders,gas,...,automatic,NaN,4wd,mid-size,pickup,white,"92 toyota 4x4 king cab automatic, v6, JBL blu...",ca,33.1039,-117.0770
170856,7120778854,https://sandiego.craigslist.org/ssd/ctd/d/san-...,san diego,1000,NaN,undefined,undefined,undefined,undefined,gas,...,automatic,LOHMT0023HCN00488,undefined,undefined,undefined,undefined,Auto Auction of San Diego address: 5801 Fai...,ca,32.7808,-117.1010
333060,7107691638,https://lakeland.craigslist.org/cto/d/babson-p...,lakeland,3800,NaN,undefined,oor,good,8 cylinders,gas,...,automatic,NaN,4wd,mid-size,truck,white,Nice truck 4wD. Everything works new brakes 80...,fl,27.7618,-81.5749


In [171]:
for elem in df[df['year'].isnull()]['description']:
    print(elem)
    print('\n')

03 jeep liberty 2 wheel drive v6 runs good cold air and hot heat all windows work real nice on the inside and out it does have a reconstructed title 423twofivetwosevenzeroninesix


92 toyota 4x4 king cab automatic, v6, JBL  blue tooth stereo, clean title, 3rd owner, no accident, runs great


Auto Auction of San Diego address:    5801 Fairmount Ave Mission Valley CA, 92120phone:      ☎ (619) 430-4345 ext 32text:       Text 32 to (619) 430-4345 for more details  link:       http://inventory.autoauctionofsandiego.com/v/39002970/2/21825851/32  contact:    Auto Auction of San Diego      PUBLIC AUCTION OPENING BID Notes from the DealerMany other cars will be auctioned off this Saturday. All cars are smogged and have clean titles. Visit www.AutoAuctionOfSanDiego.com to see more cars. Vehicle InfoStock #: 25334VIN: LOHMT0023HCN00488Condition: UsedDescription Big PUBLIC Auto Auction this Saturday 11am in Mission Valley! The OPENING BID for this vehicle is $1000. The Auto Auction is OPEN TO THE 

Pela descrição so consegue-se informação do ano para linha com indice 170030 e indice 15321, o toyota de ano 92 e o jeep 2003, respectivamente.

In [190]:
df.at[170030, 'year'] = 1992
df.at[15321, 'year'] = 2003

#### As outras 2 linhas vamos preencher com a mediana dos anos do dataset

In [186]:
df['year'].fillna(df['year'].median(), inplace=True)
df.isnull().sum()

id                   0
url                  0
region               0
price                0
year                 0
manufacturer         0
model                0
condition            0
cylinders            0
fuel                 0
odometer         75124
title_status         0
transmission         0
vin             196625
drive                0
size                 0
type                 0
paint_color          0
description          0
state                0
lat               8211
long              8211
dtype: int64

O ano foi inferido como float64 pelo pandas, verificamos se há ou não casas decimais no com a função 'get_decimal' definida no script Utils.py para cada linha do dataframe:

In [196]:
df['year'].dtype

dtype('float64')

Como a serie retornada é vazia, todos os anos são números inteiros.

In [203]:
# testa se existe algum ano do tipo 2019.2 ou 2019.1, que possa ser entendido como float
test = df['year'].apply(get_decimal)
test = test[test > 0]
test

Series([], Name: year, dtype: float64)

### Tratamento de dados ausentes em Dados do tipo object
Os dados do tipo oject são as colunas 'description', 'url' e 'vin'. Apenas o 'vin' que é o codigo identificador do veículo possui dados ausentes. Como tem muitas linhas em que o 'vin' é nulo e esse código não será tão útil no projeto, vamos preecher com o valor 'undefined'.

In [194]:
df['vin'].fillna('undefined', inplace=True)
df.isnull().sum()

id                  0
url                 0
region              0
price               0
year                0
manufacturer        0
model               0
condition           0
cylinders           0
fuel                0
odometer        75124
title_status        0
transmission        0
vin                 0
drive               0
size                0
type                0
paint_color         0
description         0
state               0
lat              8211
long             8211
dtype: int64

### Tratamento de dados ausentes em Dados do tipo Float